# 랭체인 툴 사용 예시 : 90년대 인기 애니메이션의 한국어 OST 유튜브 링크 가져오기 
추적 확인 : https://smith.langchain.com/public/d795b123-8a9b-4199-b998-f304172008eb/r

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import openai 
import os 

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5d95de64-fc49-4002-97a5-5063bd8dc66c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

# tools 

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

ddg_search = DuckDuckGoSearchRun()

ddg_search.run("Who is the CEO of NVIDIA?")

"Jensen Huang cofounded graphics-chip maker Nvidia in 1993, and has served as its CEO and president ever since. Huang owns approximately 3% of Nvidia, which went public in 1999. Born in Taiwan ... Nvidia CEO Jensen Huang 60 Minutes Jensen Huang: This is Blackwell. Designed in America, but made in Taiwan like most advanced semiconductors, Blackwell, he says, is the fastest chip ever. Nvidia CEO Jensen Huang delivers his keynote speech ahead of Computex 2024 in Taipei on June 2, 2024. When Nvidia surpassed Apple this week to become America's second most valuable company, its ... Nvidia CEO Jensen Huang is now worth $113 billion, according to the Bloomberg Billionaire Index. His fortune has ballooned over the past year following a surge in Nvidia stock. Huang is now richer ... The rise of Jensen Huang, the Nvidia CEO who was born in Taiwan, raised in Kentucky, and is now one of the richest men on earth. Jane Thier. Thu, Feb 22, 2024, 1:06 PM 4 min read."

In [4]:
from langchain_community.tools import YouTubeSearchTool

youtube_search = YouTubeSearchTool()

youtube_search.run("nvidia", 5)

['https://www.youtube.com/watch?v=TWfBbgfxarA&pp=ygUGbnZpZGlh', 'https://www.youtube.com/watch?v=K5Dwc9ttyos&pp=ygUGbnZpZGlh']

"['https://www.youtube.com/watch?v=TWfBbgfxarA&pp=ygUGbnZpZGlh', 'https://www.youtube.com/watch?v=K5Dwc9ttyos&pp=ygUGbnZpZGlh']"

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("nvidia")

"Page: Nvidia\nSummary: Nvidia Corporation (, en-VID-ee-ə) is an American multinational corporation and technology company headquartered in Santa Clara, California, and incorporated in Delaware. It is a software and fabless company which designs and supplies graphics processing units (GPUs), application programming interfaces (APIs) for data science and high-performance computing, as well as system on a chip units (SoCs) for the mobile computing and automotive market. Nvidia is also a dominant supplier of artificial intelligence (AI) hardware and software.\nNvidia's professional line of GPUs are used for edge-to-cloud computing and in supercomputers and workstations for applications in such fields as architecture, engineering and construction, media and entertainment, automotive, scientific research, and manufacturing design. Its GeForce line of GPUs are aimed at the consumer market and are used in applications such as video editing, 3D rendering and PC gaming. With a market share of 8

# Agent

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

llm = ChatOpenAI(model="gpt-4o", temperature=0)

system_prompt = """You are an LLM agent equipped with three powerful tools: ddg_search, youtube_search, and wikipedia. Each tool has specific use cases to enhance your response generation. Follow these guidelines to determine when and how to use each tool:

### ddg_earch ### 
Use Case: Use search for real-time information, current events, news articles, recent updates, or when specific knowledge about a new term, concept, or trend is required.
Example Prompt: "Search for the latest news on advancements in generative AI."

### youtube_search ### 
Use Case: Use youtube_search when the user requests videos, tutorials, music, interviews, or any visual/audio content available on YouTube.
Example Prompt: "Find a video tutorial on how to use FastAPI for backend development."

### wikipedia ### 
Use Case: Use wikipedia for comprehensive, encyclopedic information on well-established topics, historical events, biographical details, scientific concepts, and general knowledge.
Example Prompt: "Look up information on the history of the Eiffel Tower on Wikipedia."

### Instructions ### 
Always select the most appropriate tool based on the user's query.
Combine information from multiple tools when necessary to provide a comprehensive response.
Ensure that your responses are accurate, relevant, and up-to-date."""

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_prompt)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

In [8]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

tools = [ddg_search, youtube_search, wikipedia]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 

In [9]:
response = agent_executor.invoke({"input": "Find 5 popular anime from the 1990s and provide a YouTube link to the Korean OST."})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'popular anime from the 1990s'}`


Page: History of anime in the United States
Summary: The history of anime in the United States began in 1961, when Magic Boy and The White Snake Enchantress, both produced by Toei Animation, became the first and second anime films to receive documented releases in the country. Anime has since found success with a growing audience in the region, with Astro Boy often being noted as the first anime to receive widespread syndication, especially in the United States. Additionally, anime's growth in popularity in the US during the 1990s, commonly referred to as the "anime boom," is credited with much of anime's enduring relevance to popular culture outside Japan.
While many anime series have originally been distributed  by pirates and fansubbers in the past via bootleg releases from the late-1990s to mid-2000s, such practices have rapidly declined since the early-2010s due to the

In [10]:
print(response["output"])

Here are the YouTube links to the Korean OSTs for the popular 1990s anime:

1. **Neon Genesis Evangelion (1995)**
   - [Korean OST](https://www.youtube.com/watch?v=eppqcuMJchQ&pp=ygUiTmVvbiBHZW5lc2lzIEV2YW5nZWxpb24gS29yZWFuIE9TVA%3D%3D)

2. **Cowboy Bebop (1998)**
   - [Korean OST](https://www.youtube.com/watch?v=v-F-bB6c5aU&pp=ygUXQ293Ym95IEJlYm9wIEtvcmVhbiBPU1Q%3D)

3. **Sailor Moon (1992)**
   - [Korean OST](https://www.youtube.com/watch?v=qErQA12w0oA&pp=ygUWU2FpbG9yIE1vb24gS29yZWFuIE9TVA%3D%3D)

4. **Dragon Ball Z (1989-1996)**
   - [Korean OST](https://www.youtube.com/watch?v=vLanxuRgnoI&pp=ygUYRHJhZ29uIEJhbGwgWiBLb3JlYW4gT1NU)

5. **Yu Yu Hakusho (1992-1995)**
   - [Korean OST](https://www.youtube.com/watch?v=lRezaTVzkGk&pp=ygUYWXUgWXUgSGFrdXNobyBLb3JlYW4gT1NU)

Enjoy the music!
